# Test Notebook

In [1]:
import pandas as pd
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from app.utils import clean_html

In [2]:
DATA_DIR_PATH = "../data/"
LANG = "_DE"

TRAIN_PATH_CSV = DATA_DIR_PATH + "train" + LANG + ".csv"
TEST_PATH_CSV = DATA_DIR_PATH + "test" + LANG + ".csv"

In [5]:
%%time
train = pd.read_csv(TRAIN_PATH_CSV)

CPU times: user 30.3 s, sys: 1min 30s, total: 2min
Wall time: 2min 10s


In [6]:
n = 0

In [9]:
old_html = train.iloc[n]["html"]
old_html = old_html.replace("\n", "")
old_html = old_html.replace("\r", "")
old_html = old_html.replace("\t", "")
with open("../app/old/test.html", "w+") as f:
    f.write(old_html)

## real tests

In [52]:
train2 = train.head(10)

In [50]:
from lxml.html.clean import Cleaner

def clean_html_boilerplate(string):
    """ Cleans boilerplate HTML tags from HTML."""
    
    cleaner = Cleaner(page_structure=False, meta=True, style=True, kill_tags=["img"])
    clean = cleaner.clean_html(string)
    clean = clean.replace("\n", "")
    clean = clean.replace("\r", "")
    clean = clean.replace("\t", "")

    return clean

In [51]:
train2["html"] = train2.apply(lambda row: clean_html_boilerplate(row["html"]), axis=1)

<ipython-input-51-b9318d847efe>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train2["html"] = train2.apply(lambda row: clean_html_boilerplate(row["html"]), axis=1)


In [55]:
train.head(3)

,url,industry,industry_label,group,group_representative,html,text,source,country,group_representative_label
0,http://www.bruggcables.com,57,Oil & Energy,man,55,"<html lang=""de""><head><base href=""https://brug...",Home\n\nDeutsch\nEnglish\n\n\nShop\nProdukte\n...,xing,DE,Apparel & Fashion
1,http://www.famo24.de,133,Wholesale,good,133,"<html class=""no-js""><head><title>Startseite | ...",Startseite | FAMO24\n\nSchließen\nPasswort ver...,xing,DE,Hospitality
2,http://www.arcanum-energy.de,57,Oil & Energy,man,55,"<html lang=""de-DE""><head><title>Erneuerbare En...",Erneuerbare Energien - Arcanum Energy\n\nAus A...,xing,DE,Apparel & Fashion


In [56]:
train2.head(3)

,url,industry,industry_label,group,group_representative,html,text,source,country,group_representative_label
0,http://www.bruggcables.com,57,Oil & Energy,man,55,"<html lang=""de""><head><base href=""https://brug...",Home\n\nDeutsch\nEnglish\n\n\nShop\nProdukte\n...,xing,DE,Apparel & Fashion
1,http://www.famo24.de,133,Wholesale,good,133,"<html class=""no-js""><head><title>Startseite | ...",Startseite | FAMO24\n\nSchließen\nPasswort ver...,xing,DE,Hospitality
2,http://www.arcanum-energy.de,57,Oil & Energy,man,55,"<html lang=""de-DE""><head><title>Erneuerbare En...",Erneuerbare Energien - Arcanum Energy\n\nAus A...,xing,DE,Apparel & Fashion


In [36]:
with open("../app/old/test.html", "r") as f:
    old_html = f.read()

In [37]:
old_html[:100]

'<!DOCTYPE html>\n<html lang="de">\n\n<head>\n    <meta charset="utf-8">\n    <!-- This website is powered'

In [38]:
cleaned_html = cleans_html_boilerplate(old_html)

In [39]:
cleaned_html[:20]

'<html lang="de"><hea'

In [42]:
cleaned_html[-20:]

'      </body></html>'

In [48]:
from lxml import etree
from lxml import html
import numpy as np

html_file = html.fromstring(cleaned_html)

tags = [element.tag for element in html_file.iter()]
list(np.unique(tags))

['a',
 'base',
 'body',
 'br',
 'div',
 'dl',
 'footer',
 'h2',
 'h3',
 'h4',
 'head',
 'header',
 'hr',
 'html',
 'i',
 'label',
 'li',
 'nav',
 'p',
 'span',
 'time',
 'title',
 'ul']

In [ ]:
# If you want to visit all of the descendants
for element in html_file.iter():
    print(element.tag)
    
    
# Or, if you want to have a list of all the descendents
all_elements = list(html_file.iter())
#print([element.tag for element in all_elements])